## DDM Example
This example employs the model ddm_hgf. The first section simulates beliefs and responses of a putative agent.
The second section shows the use of PAM

In [2]:
# importing dependencies 

import numpy as np 
import HGF
import pandas as pd
import os,sys
import matplotlib.pyplot as plt

# importing HGF library
import HGF.hgf_config
import HGF.hgf_fit

# Extracting the parent dir in order to append it to the python path 
parent_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))  # Adjust if needed

# Add it to the Python path
sys.path.append(parent_dir)

# importing dependencies 
from PAM.DDM.ddm_hgf_config import ddm_hgf_config
from PAM.DDM.utl.scaled_sigmoid import scaled_sigmoid
from PAM.DDM.utl.utl_wfpt import utl_wfpt


In [3]:
# load the trial list -- Specify the path -- 
u = pd.read_csv('u.csv')
u = u.values
u = u.flatten()


## Section 1: Simulate dataset

In [ ]:
# Set parameters

om2 = -4 # learning rate
a_a = 1.2 # boundary separation
a_v = 2 # drift rate
b_a = 0 # influence of beliefs on a
b_w = 0.7 # influence of beliefs on w
b_v = 0 # no influence of beliefs on v
Ter = 0.15 # non-decision time

# Simulate Beliefs
bo = HGF.hgf_fit.fitModel([], u,HGF.hgf_config.ehgf_binary_config, HGF.hgf_config.bayes_optimal_binary_config)
priormus = bo['p_prc']['p'] 
priormus[-2]=om2
esim = HGF.hgf_sim.simModel(u,HGF.hgf.ehgf_binary,priormus)

# Extract trial-wise beliefs about u
muhat = esim['traj']['mu_hat'][:,0]


Ignored trials: []
Irregular trials: []

Initializing optimization run...

Optimization terminated successfully.
         Current function value: 265.698074
         Iterations: 6
         Function evaluations: 21
         Gradient evaluations: 7


RESULTS:

Parameter estimates - perceptual model:
 mu_0: 	 [nan  0.  1.]
 sa_0: 	 [nan 0.1 1. ]
 rho: 	 [nan  0.  0.]
 ka: 	 [1. 1.]
 om: 	 [        nan -3.57818887]
 th: 	 0.6495383397499885

MODEL QUALITY:
 LME: 	 -265.41135491471664 		 (more is better)
 AIC: 	 528.1669974755891 		 (less is better)
 BIC: 	 534.7636322086852 		 (less is better)
Ignored trials: []


## Section 2: Simulate Responses

In [ ]:
# Calculate precision and trial-wise absorbing barrier
precision = scaled_sigmoid(1 / (muhat * (1 - muhat)) - 4, 1) - 0.5
a = a_a + b_a * precision

# Trial-wise starting point and drift rate
w = 0.5 + b_w * (muhat - 0.5)
v = u * (a_v + b_v * (muhat - 0.5)) - (1 - u) * (a_v + b_v * ((1 - muhat) - 0.5))

# Initialize response times and responses
rt = np.zeros(len(u))

resp = np.zeros(len(u))

# Simulate responses and response times
for n in range(len(u)):
    P1 = utl_wfpt(np.arange(0.001, 3, 0.001), -v[n], a[n], 1 - w[n])
    P2 = utl_wfpt(np.arange(0.001, 3, 0.001), v[n], a[n], w[n])
    
    P = np.concatenate([P2[::-1], P1])
    time_values = np.concatenate([-np.arange(0.001, 3, 0.001)[::-1], np.arange(0.001, 3, 0.001)])
    
    selected_time = np.random.choice(time_values, p=P / np.sum(P))
    rt[n] = abs(selected_time)
    resp[n] = 1 if selected_time > 0 else 0

# Adjust response times by adding Ter
rt += Ter
y = np.column_stack((rt, resp))


## Section 3: Fit DDM HGF


In [ ]:
# Configure Model
overwrite_option_mus = {'c_prc' : {'om' : bo['p_prc']['om'] },'c_obs': {'b_asa': 0, 'b_vsa' : 0}}
obs_model =  ddm_hgf_config

# Combine rt and resp for the current subject
y = np.column_stack((rt, resp))  

# Fit the model 
m = HGF.hgf_fit.fitModel(y, u,HGF.hgf_config.ehgf_binary_config , obs_model, opt_model=HGF.hgf_config.quasinewton_optim_config,overwrite_opt=overwrite_option_mus)


Ignored trials: []
Irregular trials: []

Initializing optimization run...

Optimization terminated successfully.
         Current function value: -189.563030
         Iterations: 14
         Function evaluations: 119
         Gradient evaluations: 17


RESULTS:

Parameter estimates - perceptual model:
 mu_0: 	 [nan  0.  1.]
 sa_0: 	 [nan 0.1 1. ]
 rho: 	 [nan  0.  0.]
 ka: 	 [1. 1.]
 om: 	 [        nan -2.59271641]
 th: 	 6.053663744150753

Parameter estimates - observation model:
 a_a: 	 1.201866238958768
 a_v: 	 2.133789105036373
 b_w: 	 0.6693798316625874
 b_a: 	 0.0
 b_v: 	 0.0
 Ter: 	 0.15085920851999746

MODEL QUALITY:
 LME: 	 181.10873183422106 		 (more is better)
 AIC: 	 -387.6451419723989 		 (less is better)
 BIC: 	 -387.9700776188333 		 (less is better)
